In [1]:
index_cd = 'KPI200'
page_n = 1
naver_index = 'https://finance.naver.com/sise/sise_index_day.nhn?code=' + index_cd + '&page=' + str(page_n)
from urllib.request import urlopen
source = urlopen(naver_index).read()
import bs4
source = bs4.BeautifulSoup(source, 'lxml')
print(source.prettify())
td = source.find_all('td')
source.find_all('table')[0].find_all('tr')[2].find_all('td')[0]
d = source.find_all('td',class_='date')[0].text
import datetime as dt
def date_format(d):
    d = str(d).replace('-','.')
    yyyy = int(d.split('.')[0])
    mm = int(d.split('.')[1])
    dd =int(d.split('.')[2])
    this_date = dt.date(yyyy,mm,dd)
    return this_date
this_close = source.find_all('tr')[2].find_all('td')[1].text
this_close = this_close.replace(',','') #쉼표(,),제거
this_close = float(this_close) # data를 숫자로 인식
p = source.find_all('td',class_='number_1')[0].text
dates = source.find_all('td', class_='date')
prices = source.find_all('td',class_='number_1')
for n in range(len(dates)): # dates 개수만큼 반복
    this_date = dates[n].text # n번째 dates 값 추출
    this_date = date_format(this_date) # 날짜 형식으로 반환
    this_close = prices[n*4].text #0,4,8,... 등 4의 배수에 해당하는 종가지수 추출
    this_close = this_close.replace(',','') # 쉼표(,) 제거
    this_close = float(this_close) # 숫자 형식으로 변환
paging = source.find('td',class_='pgRR').find('a')['href']
paging = paging.split('&')[1]
paging = paging.split('=')[1]
last_page = source.find('td', class_='pgRR').find('a')['href']
last_page = last_page.split('&')[1]
last_page = last_page.split('=')[1]
last_page = int(last_page)

<html lang="ko">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <title>
   네이버 금융
  </title>
  <link href="/css/newstock.css?20181004080316" rel="stylesheet" type="text/css"/>
  <link href="/css/common.css?20181004080316" rel="stylesheet" type="text/css"/>
  <link href="/css/layout.css?20181004080316" rel="stylesheet" type="text/css"/>
  <link href="/css/main.css?20181004080316" rel="stylesheet" type="text/css"/>
  <link href="/css/newstock2.css?20181004080316" rel="stylesheet" type="text/css"/>
  <link href="/css/newstock3.css?20181004080316" rel="stylesheet" type="text/css"/>
  <link href="/css/world.css?20181004080316" rel="stylesheet" type="text/css"/>
 </head>
 <body>
  <script src="/js/lcslog.js?20181004080316" type="text/javascript">
  </script>
  <!-- 일별시세 -->
  <div class="box_type_m">
   <h4 class="top_tlt" style="text-align:left;">
    <em>
     일별
    </em>
    시세
   </h4>
   <table cellpadding="0" cellspacing="0" class="type_1" summary="일별

In [2]:
def historical_index_naver(index_cd, page_n=1, last_page=0):   
    naver_index = 'http://finance.naver.com/sise/sise_index_day.nhn?code=' + index_cd + '&page=' + str(page_n)
    source = urlopen(naver_index).read()   # 지정한 페이지에서 코드 읽기
    source = bs4.BeautifulSoup(source, 'lxml')   # 뷰티풀 스프로 태그별로 코드 분류
    dates = source.find_all('td', class_='date')   # <td class="date">태그에서 날짜 수집   
    prices = source.find_all('td', class_='number_1')   # <td class="number_1">태그에서 지수 수집
    for n in range(len(dates)):
        if dates[n].text.split('.')[0].isdigit():
            # 날짜 처리
            this_date = dates[n].text
            this_date= date_format(this_date)
            # 종가 처리
            this_close = prices[n*4].text   # prices 중 종가지수인 0,4,8,...번째 데이터 추출
            this_close = this_close.replace(',', '')
            this_close = float(this_close)
            # 딕셔너리에 저장
            historical_prices[this_date] = this_close
    # 페이지 네비게이션
    if last_page == 0:
        last_page = source.find('td', class_='pgRR').find('a')['href']
        # 마지막페이지 주소 추출
        last_page = last_page.split('&')[1]   # & 뒤의 page=506 부분 추출
        last_page = last_page.split('=')[1]   # = 뒤의 페이지번호만 추출
        last_page = int(last_page)   # 숫자형 변수로 변환
    # 다음 페이지 호출
    if page_n < last_page:   
        page_n = page_n + 1   
        historical_index_naver(index_cd, start_date, end_date, page_n, last_page)   
    return historical_prices  

In [3]:
def historical_index_naver(historical_prices, index_cd, start_date='', end_date='', page_n=1, last_page=0):
    if start_date:   # start_date가 있으면
        start_date = date_format(start_date)   # date 포맷으로 변환
    else:    # 없으면
        start_date = dt.date.today()   # 오늘 날짜를 지정
    if end_date:   
        end_date = date_format(end_date)   
    else:
        end_date = dt.date.today()
    naver_index = 'http://finance.naver.com/sise/sise_index_day.nhn?code=' + index_cd + '&page=' + str(page_n)
    source = urlopen(naver_index).read()   # 지정한 페이지에서 코드 읽기
    source = bs4.BeautifulSoup(source, 'lxml')   # 뷰티풀 스프로 태그별로 코드 분류
    dates = source.find_all('td', class_='date')   # <td class="date">태그에서 날짜 수집   
    prices = source.find_all('td', class_='number_1')   # <td class="number_1">태그에서 지수 수집
    for n in range(len(dates)):
        if dates[n].text.split('.')[0].isdigit():
            # 날짜 처리
            this_date = dates[n].text
            this_date= date_format(this_date)
            if this_date <= end_date and this_date >= start_date:
            # start_date와 end_date 사이에서 데이터 저장
                # 종가 처리
                this_close = prices[n*4].text   # prices 중 종가지수인 0,4,8,...번째 데이터 추출
                this_close = this_close.replace(',', '')
                this_close = float(this_close)
                # 딕셔너리에 저장
                historical_prices[this_date] = this_close
            elif this_date < start_date:   
            # start_date 이전이면 함수 종료
                return historical_prices              
    # 페이지 네비게이션
    if last_page == 0:
        last_page = source.find('td', class_='pgRR').find('a')['href']
        # 마지막페이지 주소 추출
        last_page = last_page.split('&')[1]   # & 뒤의 page=506 부분 추출
        last_page = last_page.split('=')[1]   # = 뒤의 페이지번호만 추출
        last_page = int(last_page)   # 숫자형 변수로 변환
    # 다음 페이지 호출
    if page_n < last_page:   
        page_n = page_n + 1   
        historical_index_naver(ks200, index_cd, start_date, end_date, page_n, last_page)   
    return historical_prices  

In [ ]:
index_cd = 'KOSPI'
ks11 = dict()
historical_index_naver(ks11, index_cd, '2010-1-1', '2018-10-1')
ks11

In [ ]:
code = 'KPI200'

def get_url(code):
    url = 'https://finance.naver.com/sise/sise_index_day.nhn?code={code}'.format(code=code)
    return url

def get_data(code):
    df = pd.DataFrame()
    url = get_url(code)
    for page in range(1,526):
        pg_url = '{url}&page={page}'.format(url=url, page=page)
        df = df.append(pd.read_html(pg_url, header=0)[0], ignore_index=True)
    df = df.dropna()
    return df

In [ ]:
ks200 = get_data('KPI200')